# **Manifold Learning**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import scipy.io


from sklearn.decomposition import PCA
from sklearn import manifold

In [ ]:
pd.options.display.max_columns = 7

mat = scipy.io.loadmat('../data/face_data.mat')
df = pd.DataFrame(mat['images']).T

num_images, num_pixels = df.shape
pixels_per_dimension = int(math.sqrt(num_pixels))

# 이미지 회전
for idx in df.index:
    df.loc[idx] = df.loc[idx].values.reshape(pixels_per_dimension, 
                                             pixels_per_dimension).T.reshape(-1)
    
# 첫 5행 출력
print(df.head())
print("")
print("df shape :",df.shape)

In [ ]:
list_df = np.array(df.values.tolist())
fig, ax = plt.subplots(2, 3, subplot_kw=dict(xticks=[], yticks=[]))
for i, axi in enumerate(ax.flat):
    axi.imshow(list_df[100*i].reshape(64,64), cmap='gray_r')

In [ ]:
def manifold_plot(name, origin_df, manifold_2D):
    
    fig = plt.figure()
    fig.set_size_inches(10, 10)
    ax = fig.add_subplot(111)
    ax.set_title('2D Components from %s of Facial Images'%name)
    ax.set_xlabel('Component: 1')
    ax.set_ylabel('Component: 2')

    # Show 40 of the images ont the plot
    x_size = (max(manifold_2D['Component 1']) - min(manifold_2D['Component 1'])) * 0.08
    y_size = (max(manifold_2D['Component 2']) - min(manifold_2D['Component 2'])) * 0.08
    for i in range(40):
        img_num = np.random.randint(0, num_images)
        x0 = manifold_2D.loc[img_num, 'Component 1'] - (x_size / 2.)
        y0 = manifold_2D.loc[img_num, 'Component 2'] - (y_size / 2.)
        x1 = manifold_2D.loc[img_num, 'Component 1'] + (x_size / 2.)
        y1 = manifold_2D.loc[img_num, 'Component 2'] + (y_size / 2.)

        img = origin_df.iloc[img_num,:].values.reshape(pixels_per_dimension, pixels_per_dimension)
        ax.imshow(img, aspect='auto', cmap=plt.cm.gray, 
                  interpolation='nearest', zorder=100000, extent=(x0, x1, y0, y1))

    # Show 2D components plot
    ax.scatter(manifold_2D['Component 1'], manifold_2D['Component 2'], marker='.',alpha=0.7)

    ax.set_ylabel('Up-Down Pose')
    ax.set_xlabel('Right-Left Pose')

    plt.show()

## **1. PCA 결과 확인**

In [ ]:
pca = PCA(n_components = 2)
pca.fit(df) # fit : pca 학습 함수로 주성분 2개 찾는 함수

pca_2D = pca.transform(df) # transform : 새로운 주성분으로 데이터 변환
pca_2D = pd.DataFrame(pca_2D, columns = ['Component 1', 'Component 2'])

pca_2D.head()

In [ ]:
manifold_plot("PCA", df, pca_2D)

***

## **2. ISOMAP(n=5) 결과 확인**

In [ ]:
iso = manifold.Isomap(n_neighbors=5, n_components=2)
iso.fit(df)
isomap_2Da = iso.transform(df)
isomap_2D = pd.DataFrame(isomap_2Da, columns=['Component 1', 'Component 2'])

# Left with 2 dimensions
isomap_2D.head()

In [ ]:
manifold_plot("ISOMAP(n=5)", df, isomap_2D)

***

## **3. LLE(n=5) 결과 확인**

In [ ]:
LLE = manifold.LocallyLinearEmbedding(n_neighbors=5, n_components=2)
LLE.fit(df)
LLE_manifold_2D_ = LLE.transform(df)
LLE_manifold_2D = pd.DataFrame(LLE_manifold_2D_, columns=['Component 1', 'Component 2'])

# Left with 2 dimensions
LLE_manifold_2D.head()

In [ ]:
manifold_plot("LLE", df, LLE_manifold_2D)

***

## **4. TSNE 결과 확인**

In [ ]:
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
tsne.fit(df)
tsne_2Da = tsne.fit_transform(df)
tsne_2D = pd.DataFrame(tsne_2Da, columns=['Component 1', 'Component 2'])

# Left with 2 dimensions
tsne_2D.head()

In [ ]:
manifold_plot("TSNE", df, tsne_2D)